Below is it a successfull atempt of building a customized tool to be added to a langchain agent in order to implement a simple calculator

In [1]:
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import openai
import logging
import os

In [2]:
@tool
def add(a: int, b: int) -> int:
    """Adds two numbers together."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


@tool
def square(a) -> int:
    """Calculates the square of a number."""
    a = int(a)
    return a * a


toolkit = [add, multiply, square]

In [6]:
LOG = logging.getLogger(__name__)

def setApiKey(filename="~/.openaikey.txt"):
    """Set the OpenAI API key from a file.

    Set the OpenAI API key from a file. The file should contain a single line
    with the API key. The file name can be specified as an argument. If the
    API key is already set, it will be overwritten, with a warning issues.

    Parameters
    ----------
    filename : `str`
        Name of the file containing the API key.
    """

    currentKey = os.getenv('OPENAI_API_KEY')
    if currentKey:
        LOG.warning(f"OPENAI_API_KEY is already set. Overwriting with key from {filename}")

    filename = os.path.expanduser(filename)
    with open(filename, 'r') as file:
        apiKey = file.readline().strip()

    openai.api_key = apiKey
    os.environ["OPENAI_API_KEY"] = apiKey

In [7]:
setApiKey()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
          You are a mathematical assistant. Use your tools to answer questions.
          If you do not have a tool to answer the question, say so. 
        
          Return only the answers. e.g
          Human: What is 1 + 1?
          AI: 2
          """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [9]:
agent = create_openai_tools_agent(llm, toolkit, prompt)
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

In [10]:
result = agent_executor.invoke({"input": "what is 1 + 3?"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 1, 'b': 3}`


44

> Finished chain.
4


Below there is a request to get wind data for a given date. This is related to the tool to be implemented for astrochat.

In [9]:
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
from lsst.daf.butler import Butler

import asyncio
await asyncio.sleep(1)
import nest_asyncio
nest_asyncio.apply()

client = makeEfdClient()
butler = Butler('embargo_or4', instrument='LSSTComCamSim', collections='LSSTComCamSim/raw/all')

topic = 'lsst.sal.ESS.airFlow'
dayObs = 20240625
seqNum = 123

where = f"exposure.day_obs={dayObs} AND exposure.seq_num={seqNum} AND instrument='LSSTComCamSim'"
records = list(butler.registry.queryDimensionRecords('exposure', where=where))
record = records[0]

windData = getEfdData(client, topic, expRecord=record)

In [14]:
print(type(windData))
print(windData.columns.tolist())
print(windData)

<class 'pandas.core.frame.DataFrame'>
['direction', 'directionStdDev', 'location', 'maxSpeed', 'private_efdStamp', 'private_identity', 'private_kafkaStamp', 'private_origin', 'private_rcvStamp', 'private_revCode', 'private_seqNum', 'private_sndStamp', 'salIndex', 'sensorName', 'speed', 'speedStdDev', 'timestamp']
                                   direction  directionStdDev       location  \
2024-06-26 00:56:16.214350+00:00   24.679869         2.237279  Weather tower   
2024-06-26 00:56:17.494557+00:00   30.289766         1.928553  Weather tower   
2024-06-26 00:56:18.774672+00:00   34.180000         0.203961  Weather tower   
2024-06-26 00:56:20.054118+00:00   33.710106         1.169140  Weather tower   
2024-06-26 00:56:21.175032+00:00  165.620972         7.982798        AT Dome   
2024-06-26 00:56:21.334244+00:00   29.139891         1.448260  Weather tower   
2024-06-26 00:56:22.614693+00:00   33.705002         0.233398  Weather tower   
2024-06-26 00:56:23.893923+00:00   32.215073 

In [8]:
from lsst.summit.extras.astrochat import setApiKey
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

setApiKey()

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", n=2, best_of=2)

question = "What is the answer of the meaning of life?"
joke = "Tell me a joke about cosmology?"

with get_openai_callback() as cb:
    result_1 = llm.invoke(question)
    print(result_1)
    print(cb)

    result_2 = llm.invoke(joke)
    print(result_2)
    print(cb)



The meaning of life is a highly debated and subjective topic, and there is no one definitive answer. Some people believe that the meaning of life is to find happiness and fulfillment, others believe it is to fulfill a certain purpose or destiny, and still others believe it is to seek spiritual enlightenment or connection with a higher power. Ultimately, the answer to the meaning of life may differ for each individual and can change throughout one's lifetime. 
Tokens Used: 170
	Prompt Tokens: 10
	Completion Tokens: 160
Successful Requests: 1
Total Cost (USD): $0.000335


Why did the astronomer break up with the physicist?

Because they had no chemistry, it was just a matter of time and space.
Tokens Used: 228
	Prompt Tokens: 18
	Completion Tokens: 210
Successful Requests: 2
Total Cost (USD): $0.000447
